In [1]:
from google.colab import drive 
drive.mount('gdrive')

Mounted at gdrive


In [2]:
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers, models, backend as K, callbacks
from keras.layers import LSTM
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd
from pprint import pprint
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras import layers, models, backend as K, callbacks
from keras.layers import LSTM
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from keras import regularizers
from keras.layers import TimeDistributed
import numpy

In [3]:
stock_market_df = pd.read_csv('/content/gdrive/MyDrive/Stock_market_label_data.csv')
stock_market_df['Match'] = stock_market_df['Match'].replace(['-1'], '0')
stock_market_df.drop(columns=['Company_Name', 'Unnamed: 0', 'Date', 'Name'], inplace=True)
stock_market_df.fillna(0)
stock_market_df = stock_market_df[: 15862]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (86) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
columns = stock_market_df.columns

In [5]:
stock_market_df_1 = stock_market_df.drop(columns=['Match'])
X=stock_market_df_1.values

In [ ]:
norm = MinMaxScaler()
X_norm = norm.fit_transform(stock_market_df)
X_norm = pd.DataFrame(X_norm, columns = columns)
X_norm.head()

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(stock_market_df)

In [8]:
train_size = int(len(dataset) * 0.67)
test_size = len(dataset) - train_size
train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]
print(len(train), len(test))

10627 5235


In [ ]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return numpy.array(dataX), numpy.array(dataY)

In [ ]:
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

In [ ]:
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))

In [ ]:
model = tf.keras.Sequential()
model.add(layers.LSTM(4, input_shape=(1, look_back), return_sequences=True))
model.add(TimeDistributed(layers.Dense(1, activation='sigmoid')))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = [tf.keras.metrics.AUC()])


#model.fit(trainX, trainY, epochs=100, batch_size=1, verbose=2)

In [ ]:
checkpoint_filepath = '/content/sample_data/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max')
history = model.fit(trainX,trainY,validation_data=(testX, testY),epochs=100, batch_size=1, verbose=2, callbacks=[model_checkpoint_callback])
